Import the libraries needed

In [1]:
import pandas as pd
#from pandas.compat import StringIO
from pandas.tseries.offsets import DateOffset

from datetime import datetime
import psutil

import numpy as np
import os
import pyodbc
pyodbc.drivers()

import csv

from sklearn.preprocessing import StandardScaler
import seaborn as sns

pd.options.display.max_rows = 1000

# For the Yahoo Finance api
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() 


C:\Users\m033\Anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (3.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


### Database connections and SQL to retrieve the various data needed.

In [2]:
SQL_server = 'WINPRDAI1572\CLESQLDW'
Reporting_database = 'CLE_Reporting'

pd.set_option('max_row', None)
#pd.options.display.max_rows = 1000

# Set a value for the home folder.
home_folder = "."

# Set values for the various paths.
input_path = home_folder + "\data"
external_data = home_folder + "\external_data"


print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')
os.path.abspath(os.getcwd())

2024-07-20 14:04:48.902594
svmem(total=16756752384, available=4058550272, percent=75.8, used=12698202112, free=4058550272)


'C:\\Users\\m033\\OneDrive - GWLE\\Fund_Switch_Thesis\\Pre_Synthesisation'

# 1. Get supplementary client features.

  -  Identified in Lit Review.# 1. Read back the population with switches data.

In [3]:
# READ IN directly from DW
def customer_data(server, database):
    conn = pyodbc.connect(
         'DRIVER={ODBC Driver 17 for SQL Server};'
         'SERVER='+server+';'
         'DATABASE='+database+';'
         'Trusted_Connection=yes;')
  
    query = """SELECT pol.Policy_Number
      ,pol.Risk_Commencement_Date AS RiskCommDt
--      ,pol.Cover_End_Date AS CoverEndDt
      ,pa.VIP_Type_Id          AS VIPType
      ,pa.Nationality_Code     AS Nationality
      ,CASE WHEN pa.Is_Smoker = 'True' THEN 1
            ELSE 0 
       END as IsSmoker
      ,pa.Politically_Exposed_Id AS PoliticallyExposed
      ,pa.Birth_Place               As BirthPlace
      ,a.Postcode                   AS AddressPostcode
	  ,pa.Sex_Id                    AS Gender
	  ,pa.Marital_Status_Id         AS MaritalStatus
	  ,pla.Pension_Disability_Level AS PensionDisability
	  ,pla.WOP_Disability_Level     AS WOPDisability
      ,a.City                       AS AddressCity
	  ,pa.Date_Of_Birth             AS DOB
--      ,DATEDIFF(month, GETDATE(),pol.Cover_End_Date) AS TermRemainingMths
--      ,DATEDIFF(month, GETDATE(),pol.Cover_End_Date) AS TermRemainingMths
      ,pa.Partner_Type_De           AS PartnerType
      ,pa.Is_EU_Sanctioned          AS EUSanctioned
      ,a.District                   As AddressDistrict
FROM CLT_Policy pol,
		 CLT_Product prd,
		 CLT_Policy_Life_Assured pla,
	  	 CLT_Partner pa,
		 CLT_Address a
WHERE prd.Product_Id = pol.Product_Id
AND   pla.Policy_Id = pol.Policy_Id
AND   pol.Policy_id = pla.Policy_Id
AND   pla.Partner_Id = pa.Partner_Id
AND   pa.Partner_Number = a.Partner_Number
AND   prd.Product_Code IN ('FKP_2006V1','FKP_2006V2','FKP_2008V1','FKP_2010V1','FKP_2012V1','FKP_2017V1',
                           'GNPP_2016V1','GNPP_2017V1','GNPP_2022V1','GNPP_2023V1',
                           'GNRP_2008V1','GNRP_2010V1','GNRP_2012V1','GNRP_2017V1','GNRP_2022V1',
                           'XRPP_2005V1','XRPP_2008V1','XRPP_2010V1','XRPP_2012V1','XRPP_2015V1');"""    
    
    
    return pd.read_sql(query, conn)

df_customer = customer_data(SQL_server, Reporting_database)

# Types and row count
# print stats.
print(len(df_customer.index))
now = datetime.now()
print(now)
print('==========================')

print(df_customer.dtypes)
df_customer.head(5)

137324
2024-07-20 14:04:59.265607
Policy_Number          object
RiskCommDt             object
VIPType                 int64
Nationality            object
IsSmoker                int64
PoliticallyExposed      int64
BirthPlace             object
AddressPostcode        object
Gender                  int64
MaritalStatus           int64
PensionDisability     float64
WOPDisability         float64
AddressCity            object
DOB                    object
PartnerType            object
EUSanctioned             bool
AddressDistrict        object
dtype: object


,Policy_Number,RiskCommDt,VIPType,Nationality,IsSmoker,PoliticallyExposed,BirthPlace,AddressPostcode,Gender,MaritalStatus,PensionDisability,WOPDisability,AddressCity,DOB,PartnerType,EUSanctioned,AddressDistrict
0,6708910H,2018-03-01,0,DE,0,2,München,83646,2,2,0.0,0.0,Bad Tölz,1996-10-01,Standard,False,0
1,6708915W,2018-04-01,0,DE,0,0,Marsberg,34431,1,1,0.0,0.0,Marsberg,1989-05-17,Standard,False,0
2,6708927E,2018-02-01,0,DE,0,2,Mainburg,84106,2,1,0.0,0.0,Volkenschwand,1977-07-28,Standard,False,0
3,6722995P,2018-05-01,0,GR,0,0,Trikala,81829,1,0,0.0,0.0,München,1987-03-17,Standard,False,0
4,6709066A,2018-03-01,0,DE,0,0,Bühl - Baden,22301,2,1,0.0,0.0,Hamburg,1985-02-02,Standard,False,0


## Drop obsolete or repeated features.

## Check for nulls

In [4]:
df_customer.dtypes

Policy_Number          object
RiskCommDt             object
VIPType                 int64
Nationality            object
IsSmoker                int64
PoliticallyExposed      int64
BirthPlace             object
AddressPostcode        object
Gender                  int64
MaritalStatus           int64
PensionDisability     float64
WOPDisability         float64
AddressCity            object
DOB                    object
PartnerType            object
EUSanctioned             bool
AddressDistrict        object
dtype: object

In [5]:
df_customer.isna().any()

Policy_Number         False
RiskCommDt            False
VIPType               False
Nationality           False
IsSmoker              False
PoliticallyExposed    False
BirthPlace            False
AddressPostcode       False
Gender                False
MaritalStatus         False
PensionDisability      True
WOPDisability          True
AddressCity           False
DOB                    True
PartnerType           False
EUSanctioned          False
AddressDistrict       False
dtype: bool

## Replace any nulls before checking the distributions of values for each column.

In [6]:
df_customer['PensionDisability'] = df_customer['PensionDisability'].fillna(0).apply(np.int64)
df_customer['WOPDisability'] = df_customer['WOPDisability'].fillna(0).apply(np.int64)


## Check the distributions of values for each column - ignore dates and postcode.

In [7]:
print('VIPType            : ',df_customer['VIPType'].value_counts())
print('MaritalStatus      : ',df_customer['MaritalStatus'].value_counts())
print('IsSmoker           : ',df_customer['IsSmoker'].value_counts())
print('PoliticallyExposed : ',df_customer['PoliticallyExposed'].value_counts())
print('Gender             : ',df_customer['Gender'].value_counts())
print('MaritalStatus      : ',df_customer['MaritalStatus'].value_counts())
print('PensionDisability  : ',df_customer['PensionDisability'].value_counts())
print('WOPDisability      : ',df_customer['WOPDisability'].value_counts())
print('PartnerType        : ',df_customer['PartnerType'].value_counts())
print('PensionDisability  : ',df_customer['PensionDisability'].value_counts())
print('EUSanctioned       : ',df_customer['EUSanctioned'].value_counts())
print('Nationality        : ',df_customer['Nationality'].value_counts())


VIPType            :  0    137106
2       137
3        45
1        36
Name: VIPType, dtype: int64
MaritalStatus      :  1    54257
2    40943
0    36937
3     4515
4      660
5       12
Name: MaritalStatus, dtype: int64
IsSmoker           :  0    133933
1      3391
Name: IsSmoker, dtype: int64
PoliticallyExposed :  2    88098
0    49194
1       32
Name: PoliticallyExposed, dtype: int64
Gender             :  1    76629
2    60645
0       49
3        1
Name: Gender, dtype: int64
MaritalStatus      :  1    54257
2    40943
0    36937
3     4515
4      660
5       12
Name: MaritalStatus, dtype: int64
PensionDisability  :  0    137323
1         1
Name: PensionDisability, dtype: int64
WOPDisability      :  0    137306
1        18
Name: WOPDisability, dtype: int64
PartnerType        :  Standard    137324
Name: PartnerType, dtype: int64
PensionDisability  :  0    137323
1         1
Name: PensionDisability, dtype: int64
EUSanctioned       :  False    108993
True      28331
Name: EUSanctioned, d

## Set the Postcode to int to facilitate the join.

In [8]:
df_customer['AddressPostcode'] = (
    pd.to_numeric(df_customer['AddressPostcode'],
                  errors='coerce')
      .fillna(0)
    )
df_customer['AddressPostcode'] = df_customer['AddressPostcode'].astype(int)
df_customer.dtypes

Policy_Number         object
RiskCommDt            object
VIPType                int64
Nationality           object
IsSmoker               int64
PoliticallyExposed     int64
BirthPlace            object
AddressPostcode        int32
Gender                 int64
MaritalStatus          int64
PensionDisability      int64
WOPDisability          int64
AddressCity           object
DOB                   object
PartnerType           object
EUSanctioned            bool
AddressDistrict       object
dtype: object

# Read in the postcode data.

In [9]:
# giving directory name
filename = external_data + '\Postcode_State_Metadata.xlsx'

#df_Postcode = pd.read_excel(filename,sheet_name='Tabelle1')
df_Postcode = pd.read_csv(filename)
df_Postcode['Postal_Code'] = df_Postcode['Postal_Code'].astype(int)

print(df_Postcode.dtypes)
df_Postcode.head(2)

Postal_Code          int32
Postal_Code_Name    object
District            object
State               object
dtype: object


,Postal_Code,Postal_Code_Name,District,State
0,64754,Eberbach,Landkreis Rhein-Neckar-Kreis,Baden-W�rttemberg
1,68159,Mannheim,Mannheim,Baden-W�rttemberg


# Merge the address and the external postcode data.

In [10]:
# Left Join the list of campaign contract numbers to the main dataset.
df_customer = pd.merge(df_customer, df_Postcode, left_on = ['AddressPostcode'], right_on = ['Postal_Code'] , how = 'left')

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

df_customer.head(5)

2024-07-20 14:05:00.297779
svmem(total=16756752384, available=3886981120, percent=76.8, used=12869771264, free=3886981120)


,Policy_Number,RiskCommDt,VIPType,Nationality,IsSmoker,PoliticallyExposed,BirthPlace,AddressPostcode,Gender,MaritalStatus,...,WOPDisability,AddressCity,DOB,PartnerType,EUSanctioned,AddressDistrict,Postal_Code,Postal_Code_Name,District,State
0,6708910H,2018-03-01,0,DE,0,2,München,83646,2,2,...,0,Bad Tölz,1996-10-01,Standard,False,0,83646.0,Bad T�lz,Landkreis Bad T�lz-Wolfratshausen,Bavaria
1,6708915W,2018-04-01,0,DE,0,0,Marsberg,34431,1,1,...,0,Marsberg,1989-05-17,Standard,False,0,34431.0,Marsberg,Landkreis Hochsauerlandkreis,North Rhine-Westphalia
2,6708927E,2018-02-01,0,DE,0,2,Mainburg,84106,2,1,...,0,Volkenschwand,1977-07-28,Standard,False,0,84106.0,Volkenschwand,Landkreis Kelheim,Bavaria
3,6722995P,2018-05-01,0,GR,0,0,Trikala,81829,1,0,...,0,München,1987-03-17,Standard,False,0,81829.0,Munich,M�nchen,Bavaria
4,6709066A,2018-03-01,0,DE,0,0,Bühl - Baden,22301,2,1,...,0,Hamburg,1985-02-02,Standard,False,0,22301.0,Hamburg,Hamburg,Hamburg


# Merge in the sinus milieus data.

In [11]:
# giving directory name
filename = external_data + '\sinus_milieu.csv'


df_Sinus_Milieus = pd.read_csv(filename)

print(df_Sinus_Milieus.dtypes)
df_Sinus_Milieus.head(2)

CONTRACT_NUMBER                 object
DATA_VERSION                   float64
POLICY_NUMBER                   object
VN_MILIEU_CODE                  object
VP_MILIEU_CODE                  object
VN_TARGET_GROUP_CODE            object
VP_TARGET_GROUP_CODE            object
VN_MILIEU_DESCRIPTION           object
VN_TARGET_GROUP_DESCRIPTION     object
VP_MILIEU_DESCRIPTION           object
VP_TARGET_GROUP_DESCRIPTION     object
dtype: object


,CONTRACT_NUMBER,DATA_VERSION,POLICY_NUMBER,VN_MILIEU_CODE,VP_MILIEU_CODE,VN_TARGET_GROUP_CODE,VP_TARGET_GROUP_CODE,VN_MILIEU_DESCRIPTION,VN_TARGET_GROUP_DESCRIPTION,VP_MILIEU_DESCRIPTION,VP_TARGET_GROUP_DESCRIPTION
0,6097825M,2023.0,6097825M,04_EPE,Expeditive,Selbstoptimierer,Selbstoptimierer,04_EPE,Expeditive,Selbstoptimierer,Selbstoptimierer
1,6098944M,2023.0,6098944M,10_NOK,Neo-Ökologische,Selbstoptimierer,Selbstoptimierer,10_NOK,Neo-Ökologische,Selbstoptimierer,Selbstoptimierer


## Drop all non predictive features and obsolette or repeated features and write to a csv file.

In [12]:
df_Sinus_Milieus = df_Sinus_Milieus.drop(['DATA_VERSION','POLICY_NUMBER','VP_MILIEU_CODE','VP_TARGET_GROUP_CODE','VN_TARGET_GROUP_DESCRIPTION','VP_TARGET_GROUP_DESCRIPTION'],axis=1)

# Write out to a comma separated values file.

# giving directory name
filename = input_path + '\SinusMilieus_data.csv'
df_Sinus_Milieus.to_csv(filename, encoding='utf-8', index=False)        
df_Sinus_Milieus.head(10)

,CONTRACT_NUMBER,VN_MILIEU_CODE,VN_TARGET_GROUP_CODE,VN_MILIEU_DESCRIPTION,VP_MILIEU_DESCRIPTION
0,6097825M,04_EPE,Selbstoptimierer,04_EPE,Selbstoptimierer
1,6098944M,10_NOK,Selbstoptimierer,10_NOK,Selbstoptimierer
2,6100350A,06_NOB,PreissensPragm,06_NOB,PreissensPragm
3,6102573C,01_KOG,Traditionelle,01_KOG,Traditionelle
4,6103528Y,01_KOG,Traditionelle,01_KOG,Traditionelle
5,6103662L,04_EPE,Selbstoptimierer,04_EPE,Selbstoptimierer
6,6106404L,02_PMA,AnsprRoutinierte,02_PMA,AnsprRoutinierte
7,6107203W,NaN,NaN,NaN,NaN
8,6107658A,10_NOK,BerIntOrientierung,10_NOK,BerIntOrientierung
9,6107822W,01_KOG,Selbstoptimierer,01_KOG,Selbstoptimierer


# Merge in the Sinus data.

In [13]:
df_customer = pd.merge(df_customer, df_Sinus_Milieus, left_on = ['Policy_Number'], right_on = ['CONTRACT_NUMBER'] , how = 'left') 

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

print(df_customer.dtypes)

df_customer.head(5)

2024-07-20 14:05:02.576409
svmem(total=16756752384, available=3827793920, percent=77.2, used=12928958464, free=3827793920)
Policy_Number             object
RiskCommDt                object
VIPType                    int64
Nationality               object
IsSmoker                   int64
PoliticallyExposed         int64
BirthPlace                object
AddressPostcode            int32
Gender                     int64
MaritalStatus              int64
PensionDisability          int64
WOPDisability              int64
AddressCity               object
DOB                       object
PartnerType               object
EUSanctioned                bool
AddressDistrict           object
Postal_Code              float64
Postal_Code_Name          object
District                  object
State                     object
CONTRACT_NUMBER           object
VN_MILIEU_CODE            object
VN_TARGET_GROUP_CODE      object
VN_MILIEU_DESCRIPTION     object
VP_MILIEU_DESCRIPTION     object
dtype: object


,Policy_Number,RiskCommDt,VIPType,Nationality,IsSmoker,PoliticallyExposed,BirthPlace,AddressPostcode,Gender,MaritalStatus,...,AddressDistrict,Postal_Code,Postal_Code_Name,District,State,CONTRACT_NUMBER,VN_MILIEU_CODE,VN_TARGET_GROUP_CODE,VN_MILIEU_DESCRIPTION,VP_MILIEU_DESCRIPTION
0,6708910H,2018-03-01,0,DE,0,2,München,83646,2,2,...,0,83646.0,Bad T�lz,Landkreis Bad T�lz-Wolfratshausen,Bavaria,NaN,NaN,NaN,NaN,NaN
1,6708915W,2018-04-01,0,DE,0,0,Marsberg,34431,1,1,...,0,34431.0,Marsberg,Landkreis Hochsauerlandkreis,North Rhine-Westphalia,NaN,NaN,NaN,NaN,NaN
2,6708927E,2018-02-01,0,DE,0,2,Mainburg,84106,2,1,...,0,84106.0,Volkenschwand,Landkreis Kelheim,Bavaria,6708927E,10_NOK,BerIntOrientierung,10_NOK,BerIntOrientierung
3,6722995P,2018-05-01,0,GR,0,0,Trikala,81829,1,0,...,0,81829.0,Munich,M�nchen,Bavaria,NaN,NaN,NaN,NaN,NaN
4,6709066A,2018-03-01,0,DE,0,0,Bühl - Baden,22301,2,1,...,0,22301.0,Hamburg,Hamburg,Hamburg,6709066A,06_NOB,Selbstoptimierer,06_NOB,Selbstoptimierer


## Drop all non predictive features and write to Excel

In [14]:
df_customer = df_customer.drop(['AddressPostcode','CONTRACT_NUMBER'],axis=1)

# Write out to a comma separated values file.

# giving directory name
filename = input_path + '\Client_data.csv'
df_customer.to_csv(filename, encoding='utf-8', index=False)        
df_customer.head(10)

C:\Users\m033\Anaconda3\lib\site-packages\pandas\core\internals\blocks.py:2106: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


,Policy_Number,RiskCommDt,VIPType,Nationality,IsSmoker,PoliticallyExposed,BirthPlace,Gender,MaritalStatus,PensionDisability,...,EUSanctioned,AddressDistrict,Postal_Code,Postal_Code_Name,District,State,VN_MILIEU_CODE,VN_TARGET_GROUP_CODE,VN_MILIEU_DESCRIPTION,VP_MILIEU_DESCRIPTION
0,6708910H,2018-03-01,0,DE,0,2,München,2,2,0,...,False,0,83646.0,Bad T�lz,Landkreis Bad T�lz-Wolfratshausen,Bavaria,NaN,NaN,NaN,NaN
1,6708915W,2018-04-01,0,DE,0,0,Marsberg,1,1,0,...,False,0,34431.0,Marsberg,Landkreis Hochsauerlandkreis,North Rhine-Westphalia,NaN,NaN,NaN,NaN
2,6708927E,2018-02-01,0,DE,0,2,Mainburg,2,1,0,...,False,0,84106.0,Volkenschwand,Landkreis Kelheim,Bavaria,10_NOK,BerIntOrientierung,10_NOK,BerIntOrientierung
3,6722995P,2018-05-01,0,GR,0,0,Trikala,1,0,0,...,False,0,81829.0,Munich,M�nchen,Bavaria,NaN,NaN,NaN,NaN
4,6709066A,2018-03-01,0,DE,0,0,Bühl - Baden,2,1,0,...,False,0,22301.0,Hamburg,Hamburg,Hamburg,06_NOB,Selbstoptimierer,06_NOB,Selbstoptimierer
5,6709081Y,2018-03-01,0,DE,0,2,Mannheim,2,0,0,...,False,0,68307.0,Mannheim,Mannheim,Baden-W�rttemberg,NaN,NaN,NaN,NaN
6,6715755H,2018-04-06,0,DE,0,0,Lüneburg,1,1,0,...,False,0,21385.0,Amelinghausen,Landkreis L�neburg,Lower Saxony,NaN,NaN,NaN,NaN
7,6709103C,2018-03-01,0,DE,0,0,Lüneburg,1,1,0,...,False,0,21385.0,Amelinghausen,Landkreis L�neburg,Lower Saxony,10_NOK,AnsprRoutinierte,10_NOK,AnsprRoutinierte
8,6709104E,2018-03-01,0,DE,0,2,Siegen,2,1,0,...,False,0,26125.0,Oldenburg (Oldenburg),Oldenburg (Oldenburg),Lower Saxony,06_NOB,Selbstoptimierer,06_NOB,Selbstoptimierer
9,6709127W,2018-03-01,0,DE,0,0,Grünstadt,2,1,0,...,False,0,67305.0,Ramsen,Landkreis Donnersbergkreis,Rhineland-Palatinate,09_HED,PreissensPragm,09_HED,PreissensPragm


In [15]:
%who_ls DataFrame 

['df_Postcode', 'df_Sinus_Milieus', 'df_customer']

In [16]:
del df_Postcode
del df_Sinus_Milieus
del df_customer

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-07-20 14:05:07.056914
svmem(total=16756752384, available=3757047808, percent=77.6, used=12999704576, free=3757047808)
